In [1]:
import os
# Set you Private API Key here
os.environ['OPENAI_API_KEY'] = ""

In [11]:
import openai

def get_response(input_text):
    prompt = f"根據以下的英文圖片敘述，請寫一首中文古詩。詩詞需要押韻，並保持每一句的字數一致。請讓詩詞的內容豐富，並帶有深遠的意境。圖片敘述：{input_text}\n請根據這個敘述創作詩詞。"
    response = openai.chat.completions.create(
    model="gpt-4-1106-preview",
    # model = "gpt-3.5-turbo-1106",
    messages=[
        {"role": "user", "content": prompt}
    ]
    )
    return response


In [12]:
# As a reference on calculating the cost (in USD) of using the API (optional)
def get_price(response):
    price = response.usage.prompt_tokens/1000 * 0.01 + \
            response.usage.completion_tokens/1000 * 0.03 
    return price

In [15]:
import pandas as pd
import json
from tqdm import tqdm

def process_json(input_file, json_file, start_example=1001, num_examples=1000, chunk_size=1):
    df = pd.read_json(input_file)
    total_rows = df.shape[0]

    # If num_examples is not provided, process all examples starting from start_example
    if num_examples is None:
        num_examples = total_rows - start_example
    else:
        # Ensure we don't go beyond the total number of rows
        num_examples = min(num_examples, total_rows - start_example)

    data = []
    cost = 0

    # Initialize tqdm with the total number of iterations
    progress_bar = tqdm(total=num_examples, desc="Generating Data", unit="row")

    for idx, row in df.iloc[start_example:start_example + num_examples].iterrows():
        url = row['url']
        paragraph = row['paragraph']
        # Assuming get_response and get_price are defined elsewhere
        response = get_response(paragraph)
        answer = response.choices[0].message.content
        cost += get_price(response)

        example_data = {
            'url': url,
            'paragraph': paragraph,
            'answer': answer
        }
        data.append(example_data)
        
        # Update the progress bar description and refresh the display
        progress_bar.set_description(f"Processing CSV - Cost: {cost:.2f}")
        progress_bar.update(1)

        if (idx + 1) % chunk_size == 0 or idx == total_rows - 1:
            # Write to JSON file every chunk_size examples or on the last example
            with open(json_file, 'w', encoding='utf-8') as jsonfile:
                json.dump(data, jsonfile, ensure_ascii=False, indent=2)
    
    # Close the progress bar
    progress_bar.close()


In [16]:

# Change Input file to the output path from InstructBLIP
input_file = './112-1_ADL_Final/InstructBLIP/InstructBlip_ver2.json'
output_file = 'new_gpt4_1k.json'
process_json(input_file, output_file)

Processing CSV - Cost: 5.40: 100%|██████████| 1000/1000 [2:00:53<00:00,  7.25s/row]  
